In [1]:
# usign tk backend
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import PolygonSelector
from matplotlib.path import Path
import PIL
# helper functions
edgeText = []
#vertText = []
titleText = None
def onselect(points):
    '''PolygonSelector callback: calculate distances & show edge, verts & area stats
    '''
    global vertText, edgeText, titleText
    #assert points==ps.verts
    points = np.array(points)
    n=len(points)
    distances = [ np.round(np.linalg.norm( points[i] - points[ (i+1)%n ] ),1) for i in range(n) ]
    if not edgeText is []:
        [ t.remove() for t in edgeText ]
    edgeText = [ ax.text( *(points[i]+points[ (i+1)%n ])/2 , distances[i] , color='r') for i in range(n) ]
    #if not vertText is []:
    #    [ t.remove() for t in vertText ]
    #vertText = [ ax.text( *points[i] , f"{i}:{np.round(points[i],2)}" , color='r') for i in range(n) ]
    if not titleText is None:
        titleText.remove()
    titleText = ax.text( 0,-5, f"area: {np.round(polygonArea(points),2)}" , va='top', color='r')

def polygonArea(poly):
    '''Points should be anticlockwise ordered
    '''
    n=len(poly)
    assert poly.shape[1]==2
    area=0
    for i in range(n):
        area+=np.linalg.det( [poly[i].T,poly[(i+1)%n].T] )
    a = area/2
    b = sum([ np.linalg.det( [poly[i].T,poly[(i+1)%n].T] ) for i in range(len(poly)) ])/2
    assert abs(a-b)<1e-15
    return b

def triangleArea(a,b,c):
    ''' Triangle area knowing sides
    Heron's Formula
    '''
    s = a+b+c
    return np.sqrt(s*(s-a)*(s-b)*(s-c))

# Adjust the google earth image to 1m == 1 (matplotlib data) point

In [2]:
img=PIL.Image.open('terrenobase.png')
px_width, px_height = img.size
# diagonal meters. Check "camera" legend in image
m_diag = 774
# Side meters follow the image w/h ratio
m_width = np.sqrt(m_diag**2 / ((px_height/px_width)**2+1))
m_height =np.sqrt(m_diag**2 / ((px_width/px_height)**2+1))
# Verify
# Calculated sides make a squared-angle triangle
assert abs( m_diag - np.sqrt(m_width**2+m_height**2) ) < 1e-15
# Calculated sides yield the same proportion
assert abs( m_width/m_height - px_width/px_height ) < 1e-15
#

In [3]:
# BEGIN manually adjusting
# Image position to align the bottom left corner of region of interest to 0,0
left, right, bottom, top  = -125, 541, -43, 348
# Verify the selected numbers resize the image to 1 m == 1 data point
assert right-left - m_width < 1
assert top - bottom - m_height < 1

# Show figure

In [47]:
plt.close('all')
plt.ioff()
fig, ax = plt.subplots()
imshow=ax.imshow(img, extent=(left, right, bottom, top))

# Make the Polygon Selector

In [48]:
ps = PolygonSelector(ax, onselect, useblit=False, 
                     lineprops=dict(color='red', linestyle='--', linewidth=3, alpha=0.4)
                     , markerprops=dict(marker='o', markersize=6, mec='red', mfc='white', alpha=0.3),
                     vertex_select_radius=15)

print("Select points in the figure by enclosing them within a polygon.")
print("Press the 'esc' key to start a new polygon.")
print("Try holding the 'shift' key to move all of the vertices.")
print("Try holding the 'ctrl' key to move a single vertex.")

#plt.show()

Select points in the figure by enclosing them within a polygon.
Press the 'esc' key to start a new polygon.
Try holding the 'shift' key to move all of the vertices.
Try holding the 'ctrl' key to move a single vertex.


# Helpers to get live figure data

In [ ]:
# points
ps.verts

In [ ]:
# area
np.round(polygonArea(np.array(ps.verts)),2)

In [ ]:
# distances
points=np.array(ps.verts)
[ np.round(np.linalg.norm( points[i] - points[ (i+1)%4 ] ),2) for i in range(4) ]

# Informe 3
## Reload previous polygon

In [7]:
def reload(arr):
    ps._xs, ps._ys = arr.T 
    ps.onselect(ps.verts)
    fig.canvas.draw()
    #plt.show()

In [8]:
idx = { 'SurPon':0, 'SurOri':1, 'NorOri':2, 'NorPon':3 }

In [49]:
medicion_conjunta = np.array([(0.6315442868393308, -0.14402351755704057),
                        (85.5786272609407, -3.9621101736671553),
                        (281.23482636463547, 147.70882955020787),
                        (7.920442067030843, 177.73478042519938),
                        (0.6315442868393308, -0.14402351755704057)])
reload(medicion_conjunta)

## Correccion escritura
Fe de erratas eran 301.2 y no 321.2 metros del deslinde norte.

In [50]:
plt.show()

## Symbolic functions for handling lines, distance and area equations

In [23]:
import numpy as np
import sympy as sym

In [24]:
x,y = sym.symbols('x,y')

In [25]:
def line(x,y,x1,y1,x2,y2):
    return ( y1 - y2 ) * x + ( x2 - x1 ) * y + ( x1*y2 - x2*y1 )
def slope(p1,p2):
    px1, py1 = p1
    px2, py2 = p2
    return (py2-py1)/(px2-px1)
def mline(x,y,x1,y1,m):
    return (y-y1) - m*(x-x1)
def circle(x,y,x1,y1,r):
    return (x1-x)**2+(y1-y)**2-r**2
def area(x,y,x1,y1,x2,y2,x3,y3):
    return 1/2*( x*y1 - x1*y  +
                x1*y2 - x2*y1 +
                x2*y3 - x3*y2 + 
                x3*y  -  x*y3 )

## Encontrar el punto 6 metros hacia arriba de la esquina Sur Poniente, sobre el deslinde Poniente
Es la interseccion entre:
 - la linea que define el deslinde poniente (definida por los puntos 1 y 2) 
 - Un círculo de radio 6 centrado en la esquina Sur Poniente

In [10]:
# simplificar
p = medicion_conjunta

In [ ]:
interseccionA = sym.solve(( 
    line(x,y,  x1 = p[idx['SurPon'],0], y1 = p[idx['SurPon'],1],
               x2 = p[idx['NorPon'],0], y2 = p[idx['NorPon'],1]),
    circle(x,y,x1 = p[idx['SurPon'],0], y1 = p[idx['SurPon'],1],r = 6) ), 
    (x,y))
interseccionA

In [11]:
eSurPon6 = (0.877198719103847, 5.85094553156313)
assert interseccionA[1][0] - eSurPon6[0] < 1e-15 and interseccionA[1][1] - eSurPon6[1] < 1e-15

NameError: name 'interseccionA' is not defined

## Encontrar la nueva esquina Sur Oriente
Definida sobre una línea paralela al deslinde medido en conjunto (que incluye la servidumbre), 6 metros más arriba (pasando por el punto encontrado anteriormente); hasta completar las 3,5 há.

In [ ]:
# pendiente del deslinde medido Sur
mS = slope(p[idx['SurOri']], p[idx['SurPon']])
mS

In [ ]:
interseccionB = sym.solve(( 
    mline(x,y,x1 = eSurPon6[0], y1 = eSurPon6[1], m = mS),
    area(x,y,x1 = p[idx['NorOri'],0], y1 = p[idx['NorOri'],1],
             x2 = p[idx['NorPon'],0], y2 = p[idx['NorPon'],1],
             x3 = eSurPon6[0]       , y3 = eSurPon6[1]       ) - 35000
    ),(x,y))
interseccionB

In [12]:
eSurOri6=(148.555174812880,-0.786684510387803)
assert interseccionB[x] - eSurOri6[0] < 1e-10 and interseccionB[y] - eSurOri6[1] < 1e-10

NameError: name 'interseccionB' is not defined

In [13]:
#idx = { 'SurPon':0, 'SurOri':1, 'NorOri':2, 'NorPon':3 }
propuesta = np.array([ eSurPon6,
                  eSurOri6,
                  p[idx['NorOri']],
                  p[idx['NorPon']],
                  eSurPon6 ])

In [14]:
propuesta

array([[  0.87719872,   5.85094553],
       [148.55517481,  -0.78668451],
       [281.23482636, 147.70882955],
       [  7.92044207, 177.73478043],
       [  0.87719872,   5.85094553]])

In [39]:
reload(propuesta)

In [40]:
plt.show()

## Encontrar el limite oriente de la servidumbre
Desde el deslinde Poniente

In [26]:
distancia_deslindeSur = np.linalg.norm( np.array(eSurPon6) - np.array(eSurOri6) )
distancia_deslindeSur

147.82707044289194

In [27]:
interseccionServidumbre = sym.solve(( 
    line(x,y, x1 = p[idx['SurPon'],0], y1 = p[idx['SurPon'],1],
               x2 = p[idx['SurOri'],0], y2 = p[idx['SurOri'],1]),
    circle(x,y,x1 = p[idx['SurPon'],0], y1 = p[idx['SurPon'],1],r = distancia_deslindeSur) ), 
    (x,y))
interseccionServidumbre

[(-147.046431806937, 6.49360652439389), (148.309520380615, -6.78165355950797)]

In [19]:
eSurOri = (148.309520380615, -6.78165355950797)
assert interseccionServidumbre[1][0] - eSurOri[0] < 1e-10 and interseccionServidumbre[1][1] - eSurOri[1] < 1e-10

NameError: name 'interseccionServidumbre' is not defined

In [ ]:
servidumbre = np.array([eSurOri6, eSurPon6, p[idx['SurPon']], eSurOri, eSurOri6])
reload(servidumbre)

## alargar servidumbre al poniente

In [28]:
interseccionServidumbreNorPon = sym.solve(( 
    line(x,y, x1 = eSurPon6[0], y1 = eSurPon6[1],
               x2 = eSurOri6[0], y2 = eSurOri6[1]),
    circle(x,y,x1 = eSurPon6[0], y1 = eSurPon6[1],r = distancia_deslindeSur) ), 
    (x,y))
interseccionServidumbreNorPon

[(-146.800777374672, 12.4885755735141), (148.555174812880, -0.786684510387795)]

In [30]:
servNorPon=(-146.800777374672, 12.4885755735141)

In [29]:
interseccionServidumbreSurPon = sym.solve(( 
    line(x,y, x1 = p[idx['SurPon'],0], y1 = p[idx['SurPon'],1],
               x2 = p[idx['SurOri'],0], y2 = p[idx['SurOri'],1]),
    circle(x,y,x1 = p[idx['SurPon'],0], y1 = p[idx['SurPon'],1],r = distancia_deslindeSur) ), 
    (x,y))
interseccionServidumbreSurPon

[(-147.046431806937, 6.49360652439389), (148.309520380615, -6.78165355950797)]

In [31]:
servSurPon=(-147.046431806937, 6.49360652439389)

# dibujar servidumbre

In [41]:
def lineaEntrePtos( p1, p2, ha):
    p1,p2 = np.array(p1), np.array(p2)
    l = ax.plot( *np.vstack((p1,p2)).T, color='turquoise', linestyle='--', linewidth=3, alpha=0.7)
    t = ax.text( *(p1+p2)/2, np.round(np.linalg.norm( p1 - p2),1), color='r',  ha=ha, va='center')
    fig.canvas.draw()
    return l,t

In [42]:
lr,tr = lineaEntrePtos( eSurOri6, eSurOri, 'left')
ll,tl = lineaEntrePtos( eSurPon6, p[idx['SurPon']], 'right')
#lb = ax.plot( *np.vstack((eSurOri,p[idx['SurPon']])).T, color='turquoise', linestyle='--', linewidth=3, alpha=0.7)

In [43]:
ls = ax.plot( *np.vstack((servNorPon,eSurOri6 )).T, color='turquoise', linestyle='--', linewidth=3, alpha=0.7)
ln = ax.plot( *np.vstack((servSurPon,eSurOri)).T, color='turquoise', linestyle='--', linewidth=3, alpha=0.7)

In [52]:
fig.canvas.draw()

In [46]:
ax.get_xlim(), ax.get_ylim()

((-124.53249626502597, 303.5237450143558),
 (-14.9245465278523, 191.4097527816536))

In [51]:
ax.set_xlim((-124.53249626502597, 303.5237450143558))
ax.set_ylim((-14.9245465278523, 191.4097527816536))

(-14.9245465278523, 191.4097527816536)

In [ ]:
lr[0].remove()

In [ ]:
plt.show()

In [ ]:
np.vstack((np.array(eSurOri6), np.array(eSurOri))).T, np.vstack((eSurPon6, p[idx['SurPon']])).T

## Check
 - La distancia entre la esquina Sur Poniente medida conjunta y la propuesta son 6 metros

In [ ]:
assert 6 == np.linalg.norm( propuesta[idx['SurPon']] - conjunta[idx['SurPon']] )

 - Las lineas definidas por el deslinde de la servidumbre y la propuesta son paralelas

In [ ]:
assert slope(p[idx['SurOri']], p[idx['SurPon']]) - slope(eSurPon6, eSurOri6) < 1e-15

# Dibujar la servidumbre

In [ ]:
interseccionC = sym.solve(( 
    line(x,y,  x1 = p[idx['SurPon'],0], y1 = p[idx['SurPon'],1],
               x2 = p[idx['SurOri'],0], y2 = p[idx['SurOri'],1]),
    circle(x,y,x1 = p[idx['SurPon'],0], y1 = p[idx['SurPon'],1],r = distancia_deslindeSur) ), 
    (x,y))
interseccionC

In [ ]:
servidumbre = np.array([  ])

In [ ]:
slope(eSurPon6,eSurOri6)

# Redibujo escritura

In [ ]:
escritura_informe2 = np.array([(0.6316884466397772, -0.14404252868571377),  
         (154.30300909939984, -10.15054354761331),  
         (209.51873007449916, 336.51651865862505),  
         (6.785054605092211, 87.22766713826408),
         (0.6316884466397772, -0.14404252868571377)])
#reload(escritura_informe2)

In [ ]:
x,y,x1,y1,x2,y2,x3,y3 = sym.symbols('x,y,x1,y1,x2,y2,x3,y3')

-x,y = eSurPon6
-x1,y1 -> eSurOri, slope (sSur*6), p=eSurPon6
-x2,y2 -> eNorOri, area
-x3,y3 -> eNorPon, slope (cjta*Pon), p=eSurPon6

In [ ]:
mSur = slope(eSurOri6, eSurPon6)
mPon = slope(conjunta[idx['NorPon']], conjunta[idx['SurPon']])
mSur, mPon

In [ ]:
sym.solve(( mline( x1,y1, eSurPon6[0], eSurPon6[1], mSur),
            circle( eSurPon6[0], eSurPon6[1], x1,y1, 154) ,
          ), (x1,y1))

In [ ]:
x1, y1 = (154.721878521693, -1.06385717269079)

In [ ]:
dibujoEscritura = sym.solve((
    area( eSurPon6[0], eSurPon6[1], x1,y1, x2,y2, x3,y3) - 35000,
    mline( x3,y3, eSurPon6[0], eSurPon6[1], mPon),
    circle( x1,y1, x2,y2, 351),
    circle( x2,y2, x3,y3, 301.2)
    ),( x2,y2, x3,y3))
dibujoEscritura

In [ ]:
slope(eSurOri6, eSurPon6)

In [ ]:
slope(conjunta[idx['NorPon']], conjunta[idx['SurPon']])

In [ ]:
escritura_informe3 = np.array([(0.877198719103847, 5.85094553156313),  
         (154.721878521693, -1.06385717269079),  
         (209.51873007449916, 336.51651865862505),  
         (6.785054605092211, 87.22766713826408),
         (0.877198719103847, 5.85094553156313)])
reload(escritura_informe3)

In [ ]:
triangleArea(301.2, 154, 351)

In [ ]:
145-85